# 9. Artgrams

```{sidebar} Datasets
  * [V&A](https://collections.vam.ac.uk/)
```

Graphing the popularity of a material, concept, word, etc over time can help in showing trends. This technique is commonly used with words and phrases (referred to as N-grams) showing how often a single word (or two words phrases, or three word phrases, ...) was used in some collection of published material over time.

Whilst the same could in theory be done with collection records, showing how often a word was used when an object was produced, or when it was accessioned into the museum, it would not be particularly meaningful, as the date the record was written is not the date the object was made or acquired. For example, if an object made in the 12th century says somewhere in its object record that "An X-Ray of this object shows...", this would then somewhat confusingly indicate the word "X-Ray" was used for an object made in the 12th century.

What can be shown, and does perhaps have meaning, are trends in things such as materials and techniques involved in object production over time; the changes in styles of objects accessioned into the collection over time; the depictions shown in objects over time and so on. 

```{note}
Please note whilst data visualisations can be an excellent way to see trends in data and spark new hypothesis, caution must be taken into reading too much into the figures, the data the visualisations are built on will have absences, biases and will forever remain incomplete. 
```

## Artgram for objects using glass accessioned to the collection

Let's look at objects accessioned to the V&A colletion that contain glass (picking a material at random)

In [1]:
import requests
import altair as alt
import pandas as pd

req = requests.get('https://api.vam.ac.uk/v2/objects/clusters/accession_year/search?id_material=AAT10797&cluster_size=100')
glass_df = pd.DataFrame(req.json())
glass_df.rename(columns={'value': 'Accession Year', 'count': 'Number of Objects'}, inplace=True)

line = alt.Chart(glass_df).mark_line().encode(
    x='Accession Year:T',
   y='Number of Objects:Q'
)

line.properties(title="Objects accessioned to the V&A made using glass")

alt.Chart(...)

So, a clear spike in the last few years in objects accessioned that contain glass in some form. (Spoiler - it's (glass autochrome photographs)[https://collections.vam.ac.uk/search/?page=1&id_material=AAT10797] from the RPS collection.)

## Artgram for objects depicting Rome accessioned to the collection

In [2]:
import requests
import altair as alt
import pandas as pd

req = requests.get('https://api.vam.ac.uk/v2/objects/clusters/accession_year/search?id_place=x29106&cluster_size=100')
glass_df = pd.DataFrame(req.json())
glass_df.rename(columns={'value': 'Accession Year', 'count': 'Number of Objects'}, inplace=True)

line = alt.Chart(glass_df).mark_line().encode(
    x='Accession Year:T',
   y='Number of Objects:Q'
)

line.properties(title="Objects accessioned to the V&A depicting Rome")

alt.Chart(...)

Whereas here the peak is around 1910, perhaps from an album accessioned then showing views of Rome. (Note - the id_place param will also return objects made in Rome and/or associated with Rome, at the moment the API does not have a param to query just for depictions of a place.)

## Artgram for cultural objects production

An obvious interesting question is what trends are there in the production of objects, for example, what materials and techniques were used over time (and perhaps, in what places were they used.)

However, there are two data quality issues to deal with in trying to show this data. The first is that the V&A collection is not comprehensive for all types of objects produced in all places over all time, so any conclusions over what material/techniques/etc may or may not have been popular somewhere cannot be drawn just using the subset of items in the V&A collections (for a hopeful answer to this issue, see below)

The second problem is that of uncertainly over production dates. Many historical objects do not have an exact production date, and this uncertainly over dates will increase the further back in time you go. So a 20th century object's production date may be known down to the day or month, but an object from the 14th century may only be estimated to the half-century. This could be shown on a graph using [confidence intervals](https://altair-viz.github.io/gallery/line_with_ci.html), however for us to return the information for this we would need to:
  * Retrieve all objects from each year/decade/century (depending on the timespan required)
  * For each object, retrieve each object's production date range (e.g earliest date 1st Jan 1400, latest date 31st Dec 1499)
  * Sum up all the object production ranges per year/decade/century and average them for the number of objects for each year/decade/century (e.g. for 14th century uncertainty is 45 years)
  * Return for each year/decade/century the number of objects produced and the average uncertainty over production date

(Averaging of course brings its own statistical issues that would also need to be considered)

Currently we cannot support all the processing needed to return this information from the API. Progress in this area is being made with time representation standards such as [EDTF](https://www.loc.gov/standards/datetime/), but support is still needed in database systems to implement this standard.

(N.B. an additional complication is that objects may be counted multiple times, for example if an object's production date range is the 12th to 13th century, it will be counted in both the 12th and 13th centuries results. The amount of this duplication could perhaps also be shown on the graph, but would require even more processing to return this figure)

(data-exploration-4:linked-art)=

## Artgram for objects acquired/accessioned across multiple museums

Being able to graph the difference between what (for example) Art Deco objects the V&A acquired compared to MoMa, Rijksmuseum for example could show trends in museum collecting history.  To do this we need to either have write multiple API queries against each instititions collections API (if it exists), convert the different data structures used from each to a common model, convert the values to a standard representation (e.g. dates written in different formats) and finally try to graph this.

Or, we can work on a common standard to enable this soft of research to be much simplier to carry out. This is currently in progress with [Linked Art](https://linked.art/), so hopefully we can provide an endpoint to support this soon.